# Endocrine activity prediction 

In this notebook, our goal is to train and compare the performance of three different architectures on the same task, namely predicting endocrine activity.

Endocrine activity is defined as follows:

- Active: A molecule is considered active if it shows activity in at least one nuclear receptor assay.
- Inactive: A molecule is considered inactive if it does not show activity in any such assay.
The detailed process of filtering and curating the assays can be found in the Tox21.ipynb notebook.

We overlap these assay results with our current dataset of morphological fingerprints using the corresponding CID (Compound ID). The CID for our library of over $30.000$ compounds was retrieved by querying PubChem. After preprocessing the data, we identified:

651 unique molecules with endocrine activity in the raw dataset.
642 molecules after preprocessing (based on the overlap between this notebook and the output of the notebook_1).

Out of these 642 molecules (signal assay outcome):
- 291 are active.
- 351 are inactive.

Out of these (activity assay outcome)
- We have 324 inactive 
- We have 318 active 



## Load data 

### Import Libraries

In [128]:
import pickle
import pandas as pd

In [129]:
profiles_cid_pubchem = pd.read_pickle('../../Morpheus_Flo/data/pubchem_annotation1.pkl') #pd.read_csv('../pubchem_annotation_morpho.csv',sep='\t') 
#profiles_cid_pubchem = profiles_cid_pubchem[['CID','CPD_NAME']]
print(f"There are {profiles_cid_pubchem['CID'].nunique()} unique compounds in the profiles dataset")
profiles_cid_pubchem.head(3)

There are 30397 unique compounds in the profiles dataset


,CID,CPD_NAMES
0,679,DMSO
1,2122,BRD-A56675431-001-04-0
2,3654103,BRD-A51829654-001-01-4


to do : remove dmso 

In [130]:
path_to_pickle_file ='../output_notebook_1.pkl'
profiles = pickle.load(open(path_to_pickle_file, 'rb'))
print(f"There are {len(profiles)} profiles in the morphological profiles dataset")
print(f"There are {profiles['Metadata_broad_sample'].nunique()} unique sample, and {profiles['CPD_NAME'].nunique()} unique compounds")

There are 30377 profiles in the morphological profiles dataset
There are 30377 unique sample, and 30112 unique compounds


Example of molecules with the same ‘CPD_NAME’ but different stereochemistry (enantiomers), and therefore identified with different ‘Metadata_broad_sample’ values.

In [131]:
propranolol = profiles[profiles['CPD_NAME']=='propranolol']['Metadata_broad_sample'].to_list()
profiles[profiles['CPD_NAME']=='propranolol']

,Metadata_broad_sample,CPD_NAME,CPD_SMILES,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_FormFactor,Cells_AreaShape_Orientation,Cells_AreaShape_Perimeter,...,Nuclei_Texture_InfoMeas1_RNA_10_0,Nuclei_Texture_InfoMeas1_RNA_3_0,Nuclei_Texture_InverseDifferenceMoment_ER_10_0,Nuclei_Texture_InverseDifferenceMoment_RNA_3_0,Nuclei_Texture_SumAverage_AGP_10_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_RNA_10_0,Nuclei_Texture_SumEntropy_AGP_10_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_RNA_10_0
22409,BRD-A10070317-003-05-1,propranolol,CC(C)NCC(O)COc1cccc2ccccc12,-0.181066,0.290658,-0.048351,-0.260826,-0.220636,0.144072,-0.113796,...,0.305173,0.202152,0.398321,0.281046,-0.531793,-0.102024,-0.174831,-0.199399,-0.225782,-0.385788
23179,BRD-K13994703-001-02-0,propranolol,CC(C)NC[C@H](O)COc1cccc2ccccc12,0.107945,0.042286,0.550515,0.425072,0.358772,0.319154,-0.162650,...,0.348331,0.147997,0.727870,0.199830,-0.601768,-0.308817,-0.671559,-0.340255,-0.511715,-0.482742
27230,BRD-K92830582-003-04-8,propranolol,CC(C)NC[C@@H](O)COc1cccc2ccccc12,-0.339705,0.668767,0.089364,-0.534178,0.101464,0.007005,-0.311137,...,0.211793,0.053759,-0.030465,0.220821,0.120130,0.099267,0.100746,0.257167,-0.106823,-0.212929


As required, now We have different cid for the different 'propranolol' sterochemistry 

In [132]:
profiles_cid_pubchem[profiles_cid_pubchem['CPD_NAMES'].isin(propranolol)]

,CID,CPD_NAMES
720,21138,BRD-K92830582-003-04-8
806,91536,BRD-K13994703-001-02-0
1006,4946,BRD-A10070317-003-05-1


In [133]:
profiles_cid_pubchem=profiles_cid_pubchem.rename(columns={"CPD_NAMES":"Metadata_broad_sample"})

In [134]:
profiles = profiles_cid_pubchem.merge(profiles, on='Metadata_broad_sample')
print(f"There are {profiles['CID'].nunique()} unique compounds in the activity dataset")
profiles.head(2)

There are 30163 unique compounds in the activity dataset


,CID,Metadata_broad_sample,CPD_NAME,CPD_SMILES,Cells_AreaShape_Area,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,Cells_AreaShape_Compactness,Cells_AreaShape_FormFactor,Cells_AreaShape_Orientation,...,Nuclei_Texture_InfoMeas1_RNA_10_0,Nuclei_Texture_InfoMeas1_RNA_3_0,Nuclei_Texture_InverseDifferenceMoment_ER_10_0,Nuclei_Texture_InverseDifferenceMoment_RNA_3_0,Nuclei_Texture_SumAverage_AGP_10_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_RNA_10_0,Nuclei_Texture_SumEntropy_AGP_10_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_RNA_10_0
0,679,DMSO,DMSO,CS(=O)C,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
1,2122,BRD-A56675431-001-04-0,altizide,NS(=O)(=O)c1cc2c(NC(CSCC=C)NS2(=O)=O)cc1Cl,0.618005,0.434264,0.32857,0.736717,0.575161,0.273073,...,0.331121,0.295925,0.143811,0.279064,0.29726,0.362669,0.178429,0.227307,0.306006,-0.49518


In [135]:
activity = pd.read_csv('../Data_parsed/Tox21_Endocrine_activity_BCC047.csv', sep=',')
activity.rename(columns={'PUBCHEM_CID':'CID','CPD_NAME':'Name'}, inplace=True)
print(f"There are {activity['CID'].nunique()} unique compounds in the activity dataset")
activity.head(2)

There are 651 unique compounds in the activity dataset


,CID,Endocrine_activity,CAS,SMILES,Name
0,323,inactive,91-64-5,O=C1C=Cc2ccccc2O1,Coumarin
1,338,inactive,69-72-7,OC(=O)c1ccccc1O,Salicylic acid


- We start with adding the cid columns from profiles_cid_pubchem to our pre-processed morphological profile file -> check which compounds is lost when merging the data 

In [136]:
t = activity.merge(profiles,on='CID')

In [137]:
cid_cmbd = t['CID'].to_list()
cid_activity = activity['CID'].to_list()
cid_profiles = profiles['CID'].to_list()

In [138]:
missing_cid = list(set(cid_activity) - set(cid_profiles))

In [139]:
print(f'There are {len(missing_cid)} missing CID in the merge dataset')

There are 15 missing CID in the merge dataset


In [140]:
profiles = activity.merge(profiles, on='CID')
print(f"We have {profiles['Endocrine_activity'].value_counts()['inactive']} inactive and {profiles['Endocrine_activity'].value_counts()['active']} active compounds")
profiles.head(2)

We have 324 inactive and 318 active compounds


,CID,Endocrine_activity,CAS,SMILES,Name,Metadata_broad_sample,CPD_NAME,CPD_SMILES,Cells_AreaShape_Area,Cells_AreaShape_Center_X,...,Nuclei_Texture_InfoMeas1_RNA_10_0,Nuclei_Texture_InfoMeas1_RNA_3_0,Nuclei_Texture_InverseDifferenceMoment_ER_10_0,Nuclei_Texture_InverseDifferenceMoment_RNA_3_0,Nuclei_Texture_SumAverage_AGP_10_0,Nuclei_Texture_SumAverage_DNA_10_0,Nuclei_Texture_SumAverage_RNA_10_0,Nuclei_Texture_SumEntropy_AGP_10_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_RNA_10_0
0,323,inactive,91-64-5,O=C1C=Cc2ccccc2O1,Coumarin,BRD-K23913458-001-02-5,coumarin,O=c1ccc2ccccc2o1,0.210650,0.034959,...,-0.735849,-0.312258,0.222973,0.162670,0.138114,0.460139,-0.232157,0.143794,-0.075845,-0.021427
1,338,inactive,69-72-7,OC(=O)c1ccccc1O,Salicylic acid,BRD-K93632104-001-12-3,salicylic acid,OC(=O)c1ccccc1O,0.144642,0.022497,...,0.431776,0.281568,0.371297,-0.169673,-0.063299,-0.143357,0.140593,-0.242057,-0.399327,0.110002


In [141]:

#profiles = profiles[profiles['CPD_NAME'] != 'DMSO'].reset_index(drop=True)

In [142]:
print(f"There is {profiles['CID'].nunique()},unique CID in the profiles dataset and {profiles['CAS'].nunique()} unique CAS") 

There is 636,unique CID in the profiles dataset and 636 unique CAS


## Spliting Data in Training and Testing Set

For ML, we always need training and a test set that is pairwise disjoint. <br>
To this end, we split the data into two parts, where the training set will contain 80% of the samples and the test set the remaining 20%. The training set will be utilized for training, evaluating, and optimizing the model, while the test set will be used for making predictions<br>

Moreover, we need a response vector y, i.e., the column with the activity information and a feature matrix X, i.e., a matrix with the morphological or molecular fingerprints or a combination thereof, for each compound. <br>

To make sure we do not have data leakage, we ensure that all instances of compounds, that appear several times in the dataset, are put in the same set. Moreover, we want to ensure that we have enough instances of the active class in both sets. Therefore, we perform a stratified split that ensures a constant ratio of active and inactive compounds in all datasets.

### Import library 

In [143]:

import warnings

import numpy as np
import plotly.express as px
import requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

from rdkit import Chem
from rdkit import RDLogger
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem.MolStandardize import rdMolStandardize


1. Features and variables to predict 

In [144]:
def stratified_split(data,to_remove,activity):
    data = data.drop(columns=to_remove)
    if data[activity].dtype == 'object':
        data[activity] = data[activity].apply(lambda x: 1 if x == 'active' else 0)
    X_train, X_test, y_train_activities, y_test_activities= train_test_split(
        data.iloc[:,1:], 
        data[activity].values.astype('int'), 
        random_state=42, 
        test_size=0.2, 
        shuffle=True,
        stratify=data[activity].values.astype('int')) #stratify split 
    return data, X_train, X_test, y_train_activities, y_test_activities

In [145]:
to_remove = ['CID', 'SMILES', 'CPD_NAME','Metadata_broad_sample', 'CPD_SMILES','CAS','Name']

In [146]:
data, X_train, X_test, y_train_activities, y_test_activities = stratified_split(profiles,to_remove,'Endocrine_activity')

In [147]:
np.sum( y_test_activities==1)


64

2. Features and variables to predict but make it structural 

We aim to sanitize SMILES strings by keeping the fragment parent, removing ions. Since we have a limited number of data points, we first ensure that all SMILES can be successfully converted into RDKit objects. If any conversion fails, we retrieve the appropriate isomeric structure from PubChem as a fallback

In [148]:
def get_smiles_from_cid(cid):
    '''
    We get smile for cid from pubchem, for molecule where smile is not available when turned into a rdkit object
    '''
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/CanonicalSMILES/JSON"
    
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        smiles = data['PropertyTable']['Properties'][0]['CanonicalSMILES']
        return smiles
    else:
        return f"Error: Unable to retrieve data for CID {cid}"

def replace_smile(data,old_smiles,new_smile):
    '''
    Replace the smile queried from pubchem in the dataset
    '''
    
    data.loc[data['SMILES'] == old_smiles, 'SMILES'] = new_smile
    return data

def check_rdkit_object(dataset,SMILES):
    ''' 
    Check if all smiles can be converted to rdkit object
    '''
    lg = RDLogger.logger()
    lg.setLevel(RDLogger.CRITICAL)
    object = dataset[SMILES].apply(Chem.MolFromSmiles)
    idx = object[object.isnull()].index
    for i in idx:
        old_smiles = dataset.loc[i, 'SMILES']
        CID = dataset.loc[i, 'CID']
        new_smiles = get_smiles_from_cid(CID)
        dataset= replace_smile(dataset,old_smiles,new_smiles)
    object = dataset[SMILES].apply(Chem.MolFromSmiles)
    if object.isnull().sum() == 0:
        print('All smiles can be converted to rdkit object')
    return dataset

def clean_std_smiles(dataset, smiles_column):
    """
    This function standardizes the SMILES strings in the dataset by:
    1. Removing salts or solvents.
    2. Keeping only the parent molecule.
    3. Sanitizing the molecule to ensure it's chemically valid.

    """
    
    clean_mol = [Chem.MolFromSmiles(smile) for smile in dataset[smiles_column]]
    parent_clean_mol = [rdMolStandardize.FragmentParent(mol) for mol in clean_mol]
    # Sanitize the parent molecules
    for mol in parent_clean_mol:
        if mol is not None:
            try:
                Chem.SanitizeMol(mol)
            except ValueError as e:
                print(f"Sanitization failed for molecule: {Chem.MolToSmiles(mol)}")
    
    # Convert sanitized molecules back to SMILES
    smiles = [Chem.MolToSmiles(mol) if mol is not None else None for mol in parent_clean_mol]
    dataset['STD_smile'] = smiles
    
    return dataset


a. Get the indices of the training and testing sets to split the structural data in the same way as the morphological data.

In [149]:
index_train = data.iloc[X_train.index,0]
index_test = data.iloc[X_test.index,0]
X_train_struct = profiles.loc[index_train.index, ['SMILES', 'Endocrine_activity','CID']]
X_test_struct = profiles.loc[index_test.index, ['SMILES', 'Endocrine_activity','CID']]

In [150]:
X_train_struct = check_rdkit_object(X_train_struct,'SMILES')

All smiles can be converted to rdkit object


In [151]:
X_test_struct = check_rdkit_object(X_test_struct,'SMILES')

All smiles can be converted to rdkit object


b. Standardize smile within each set 

In [152]:
print('Standardized SMILES for training set')
X_train_struct = clean_std_smiles(X_train_struct,'SMILES')
print('Standardized SMILES for testing set')
X_test_struct = clean_std_smiles(X_test_struct,'SMILES')


Standardized SMILES for training set
Standardized SMILES for testing set


c. Generate fingerprint 

Here we implement two functions : 
- One to instantiate Morgan fingerprints generator; _morgan_bin_
- One that call and computes the binary morgan fingerprint and happened it to the data frame; _make_fps_
- One to converts the structure fps features that are stored in an array

In [153]:
def morgan_bin(fpg,dataset):
    
    '''
    Generate Morgan fingerprints as binary vector  
        Parameters : 
            fpg (object): rdFingerprintGenerator(radius=X,fpSize=X) object generated by make_fps()
            dataset (data frame): data frame having a column named 'STD_smile' with smiles
        Returns : 
            dataset (data frame): same data frame with one more column named 'morganB_fps' within a list
    '''     
    RDLogger.DisableLog('rdApp.info')
    circular_bit_fp = [fpg.GetFingerprint(Chem.MolFromSmiles(smiles)).ToList() for smiles in dataset['STD_smile']]
    dataset['morganB_fps'] =  circular_bit_fp
    
    return dataset

def make_fps(dataset,val_radius=2,n_bits=1024):
    
    '''
    Make fingerprints is the function call to create structural fingerprints, combine morphological fingerprints etc 
        Parameters : 
            dataset_profiles (data frame): initial dataset having the morphological profiles  
            val_radius (int):  default = 2 can be changed, used to make morgan fingerprint cf GetMorganGenerator() Rdkit function 
            n_bits (int): default = 1024 can be changed, used to make morgan fingerprint cf GetMorganGenerator() Rdkit function 
        Returns : 
            dataset (data frame): initial dataset with a supplementary column containing the newly generated fps -> to drop before running make_models for a different fps 
    ''' 
    RDLogger.DisableLog('rdApp.info')
    fpg = rdFingerprintGenerator.GetMorganGenerator(radius=val_radius, fpSize=n_bits)
    dataset = morgan_bin(fpg,dataset)


    return dataset

def convert_list_features_to_numpy(x):
    '''
        converts the features that are stored in an array containing lists to an array of arrays such that shape works

        @param x: array containing lists 
        @return x_new: array of arrays of ints
    '''
    new_x = []
    for element in x:
        new_x.append(np.array(element))
    return np.array(new_x)

In [154]:
test = X_train_struct.copy()

In [155]:
morgan_bin(rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=1024),test)
X_train_struct_fps = convert_list_features_to_numpy(test['morganB_fps'].values)
morgan_bin(rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=1024),X_test_struct)
X_test_struct_fps = convert_list_features_to_numpy(X_test_struct['morganB_fps'].values)

In [156]:
convert_list_features_to_numpy(test['morganB_fps'].values).shape

(513, 1024)

In [157]:
def convert_array_to_df(array):
    '''
    Converts an array of arrays to a pandas dataframe

    @param array: array of fps
    '''
    num_columns = array.shape[1]
    column_names = [f'mfp_{i+1}' for i in range(num_columns)]
    fingerprint_df = pd.DataFrame(array, columns=column_names)
    return fingerprint_df


In [158]:
fingerprint_train_df = convert_array_to_df(X_train_struct_fps)
fingerprint_test_df = convert_array_to_df(X_test_struct_fps)

In [159]:
fingerprint_train_df.head(2)

,mfp_1,mfp_2,mfp_3,mfp_4,mfp_5,mfp_6,mfp_7,mfp_8,mfp_9,mfp_10,...,mfp_1015,mfp_1016,mfp_1017,mfp_1018,mfp_1019,mfp_1020,mfp_1021,mfp_1022,mfp_1023,mfp_1024
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


3. Visualisation of the training and testing set

Here we visualise the chemical and morphological space of the sets

In [160]:
def tsne_plot_activity_set(X_train, X_test, y_train_activities, y_test_activities,features_type):
    '''
    Plot the t-SNE projection of either the chemical or morphological space of the compounds 
    in the training and test sets. To see if both sets are overlapping in both spaces.
    We will change the seed in the stratified split until we get a good overlap of the two sets. 
    '''

    # Add 'activity' column to X_train using assign()
    Train = X_train.assign(activity=y_train_activities)
    Test = X_test.assign(activity=y_test_activities)

    #Add a set column to Train and Test
    Train['set'] = ['train'] * len(X_train)
    Test['set'] = ['test'] * len(X_test)

    # Combine Train and Test 
    data_combined = pd.concat([Train, Test])

    # Fit tsne on the features of the combined data
    tsne = TSNE(n_components=2, random_state=52)
    projection = tsne.fit_transform(data_combined.iloc[:,:-1])
    # Retrieve the projection coordinated for visualization and the labels
    projections_df = pd.DataFrame(projection, columns=['x', 'y'])
    projections_df['activity'] = data_combined['activity'].values.astype('str')
    projections_df['set'] = data_combined['set'].values

    # scatter plot with Plotly
    fig = px.scatter(
    projections_df, 
    x='x', 
    y='y', 
    color='activity',  
    symbol='set',
    color_discrete_sequence=[px.colors.qualitative.Dark2[4], px.colors.qualitative.Dark2[3]],
    labels={'color': 'Activity', 'symbol': 'Set'},
    title="T-SNE Plot with Train and Test Sets Overlaid of" + ' ' + features_type + ' ' + 'fingerprint.',
    )
    fig.show()
    return projections_df

In [161]:
tsne_plot_activity_set(X_train, X_test, y_train_activities, y_test_activities, "Morphological")

,x,y,activity,set
0,17.039169,8.175418,1,train
1,3.857381,3.277592,0,train
2,-18.029514,7.136495,0,train
3,16.488791,-7.566903,1,train
4,3.310899,-8.645494,0,train
...,...,...,...,...
637,-7.599357,1.407192,0,test
638,-16.534430,-9.452256,0,test
639,-21.224817,4.547368,0,test
640,11.703199,2.435776,1,test


In [162]:
tsne_plot_activity_set(fingerprint_train_df, fingerprint_test_df, y_train_activities, y_test_activities, "Structrural")

,x,y,activity,set
0,23.164110,22.224092,1,train
1,-20.373955,-5.774296,0,train
2,-23.501822,25.021399,0,train
3,18.197857,37.299217,1,train
4,-12.001395,30.482222,0,train
...,...,...,...,...
637,28.708344,-13.171158,0,test
638,-22.948133,10.442121,0,test
639,-7.087155,28.295479,0,test
640,12.659896,8.861396,1,test


## Multi layer perceptron 

### Import library 

In [163]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_validate
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import  balanced_accuracy_score,classification_report

### Cross-validation and Hyperparemeter Tuning 

### Fine tune RF using random search 

In [164]:
def rf_cross_validation(X_train, y_train, max_depth_range=[10, 20, 30, 50], num_tree_range=[100, 200, 300, 500], min_samples_leaf_range=[5,10, 15, 20, 25]):
    '''
        performs a 5-fold CV for a Random Forest for given X_train and y_train

        @param X_train: the training matrix
        @param y_train: the associated response vector
        @param max_depth_range: list containing the values that should be tested for max depth, default [10,20,30]
        @param num_tree_range: list containing the values that should be tested for the number of trees, default [100,300,500]
        @param min_samples_leaf_range: list containing the values that should be tested for the minimum number of samples per leaf, default [10,15,20]

        @return: a forest with the best hyperparameter according to the estimated test MSE and trained on the whole training set
    '''
    best_score = -float('inf')
    for depth in max_depth_range:
        cv_results = cross_validate(RandomForestClassifier(random_state=42, max_depth=depth, n_jobs=-1,
                                    class_weight='balanced'), X=X_train, y=y_train, scoring='balanced_accuracy', cv=5)
        score = np.mean(cv_results['test_score'])
        if score > best_score:
            best_score = score
            best_depth = depth

    best_score = -float('inf')
    for n_tree in num_tree_range:
        cv_results = cross_validate(RandomForestClassifier(random_state=42, n_estimators=n_tree,
                                    n_jobs=-1, class_weight='balanced'), X=X_train, y=y_train, scoring='balanced_accuracy', cv=5)
        score = np.mean(cv_results['test_score'])
        if score > best_score:
            best_score = score
            best_n_tree = n_tree

    best_score = -float('inf')
    for num_samples in min_samples_leaf_range:
        cv_results = cross_validate(RandomForestClassifier(random_state=42, min_samples_leaf=num_samples,
                                    n_jobs=-1, class_weight='balanced'), X=X_train, y=y_train, scoring='balanced_accuracy', cv=5)
        score = np.mean(cv_results['test_score'])
        if score > best_score:
            best_score = score
            best_min_samples = num_samples

    rf = RandomForestClassifier(random_state=42, n_estimators=best_n_tree, max_depth=best_depth,
                                min_samples_leaf=best_min_samples, n_jobs=-1, class_weight='balanced')
    
    return rf

In [165]:
best_estimator_morpho = rf_cross_validation(X_train, y_train_activities)

In [166]:
best_estimator_struct = rf_cross_validation(X_train_struct_fps, y_train_activities)

### Fine tune MLPC using random search 

In [167]:
def MLPC_cross_validation(X_train, y_train, 
                        hidden_layer_sizes = [50,150,250, 350, 450, 550], 
                        activations = ['logistic', 'tanh', 'relu'],
                        solvers =  ['adam', 'sgd'],
                        alphas = [0.0001, 0.001, 0.01, 0.1, 1],
                        learning_rate = ['constant', 'adaptive'],
                        max_iter = [150, 400, 600, 800],
                        batch_size = [80, 120, 180], 
                      ):
    '''
        performs a 5-fold CV for a Random Forest for given X_train and y_train

        @param X_train: the training matrix
        @param y_train: the associated response vector
        @param max_depth_range: list containing the values that should be tested for max depth, default [10,20,30]
        @param num_tree_range: list containing the values that should be tested for the number of trees, default [100,300,500]
        @param min_samples_leaf_range: list containing the values that should be tested for the minimum number of samples per leaf, default [10,15,20]

        @return: a forest with the best hyperparameter according to the estimated test MSE and trained on the whole training set
    '''
    best_score = -float('inf')
    for layer in hidden_layer_sizes :
        cv_results = cross_validate(MLPClassifier(random_state=42,hidden_layer_sizes=layer),
                                      n_jobs=1, X=X_train, y=y_train, scoring='balanced_accuracy', cv=5)
        score = np.mean(cv_results['test_score'])
        if score > best_score:
            best_score = score
            best_n_layer =  layer

    best_score = -float('inf')
    for a in alphas:
        cv_results = cross_validate(MLPClassifier(random_state=42, alpha = a), n_jobs=1, X=X_train, y=y_train, scoring='balanced_accuracy', cv=5)
        score = np.mean(cv_results['test_score'])
        if score > best_score:
            best_score = score
            best_alpha_ = a

    best_score = -float('inf')
    for solver_ in solvers:
        cv_results = cross_validate(MLPClassifier(random_state=42, solver = solver_), n_jobs=1, X=X_train, y=y_train, scoring='balanced_accuracy', cv=5)
        score = np.mean(cv_results['test_score'])
        if score > best_score:
            best_score = score
            best_solver = solver_
    
    best_score = -float('inf')
    for activation_ in activations:
        cv_results = cross_validate(MLPClassifier(random_state=42, activation= activation_),n_jobs=1,
                                     X=X_train, y=y_train, scoring='balanced_accuracy', cv=5)
        score = np.mean(cv_results['test_score'])
        if score > best_score:
            best_score = score
            best_activation= activation_
    
    best_score = -float('inf')
    for batch in batch_size:
        cv_results = cross_validate(MLPClassifier(random_state=42, batch_size= batch), n_jobs=1, X=X_train, y=y_train, scoring='balanced_accuracy', cv=5)
        score = np.mean(cv_results['test_score'])
        if score > best_score:
            best_score = score
            best_batch_size= batch
    
    best_score = -float('inf')
    for iter in max_iter:
        cv_results = cross_validate(MLPClassifier(random_state=42, max_iter = iter), n_jobs=1, X=X_train, y=y_train, scoring='balanced_accuracy', cv=5)
        score = np.mean(cv_results['test_score'])
        if score > best_score:
            best_score = score
            best_nb_iter= iter

    best_score = -float('inf')
    for lr in learning_rate:
        cv_results = cross_validate(MLPClassifier(random_state=42, learning_rate = lr), n_jobs=1, X=X_train, y=y_train, scoring='balanced_accuracy', cv=5)
        score = np.mean(cv_results['test_score'])
        if score > best_score:
            best_score = score
            best_lr = lr

    MLPC = MLPClassifier(random_state  = 42,
                         hidden_layer_sizes = best_n_layer,
                         alpha = best_alpha_,
                         solver = best_solver,
                         activation = activation_,
                         batch_size = best_batch_size,
                         max_iter= best_nb_iter,
                         learning_rate= best_lr
                         )
    
    return MLPC

In [168]:
best_estimator_morpho_mlpc = MLPC_cross_validation(X_train,y_train_activities)


/Users/flodje_uds/anaconda3/envs/mimb_cellpainting/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/Users/flodje_uds/anaconda3/envs/mimb_cellpainting/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/Users/flodje_uds/anaconda3/envs/mimb_cellpainting/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/Users/flodje_uds/anaconda3/envs/mimb_cellpainting/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conv

In [169]:
best_estimator_struct_mlpc = MLPC_cross_validation(X_train_struct_fps,y_train_activities)

/Users/flodje_uds/anaconda3/envs/mimb_cellpainting/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/Users/flodje_uds/anaconda3/envs/mimb_cellpainting/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/Users/flodje_uds/anaconda3/envs/mimb_cellpainting/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/Users/flodje_uds/anaconda3/envs/mimb_cellpainting/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conv

In [170]:
combined_train = np.concatenate([X_train.values,fingerprint_train_df.values],axis=1)
combined_test = np.concatenate([X_test.values,fingerprint_test_df.values],axis=1)
best_estimator_combined = rf_cross_validation(combined_train, y_train_activities)
best_estimator_combined_mlpc = MLPC_cross_validation(combined_train, y_train_activities)

/Users/flodje_uds/anaconda3/envs/mimb_cellpainting/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/Users/flodje_uds/anaconda3/envs/mimb_cellpainting/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/Users/flodje_uds/anaconda3/envs/mimb_cellpainting/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/Users/flodje_uds/anaconda3/envs/mimb_cellpainting/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't conv

## Model Instantiation and Evaluation Function

In [115]:
models = {
    "RF"         : [RandomForestClassifier(random_state=42,class_weight ='balanced')], ##default
    "RF_morpho"  : [best_estimator_morpho] ,
    "RF_struct"  : [best_estimator_struct],
    "RF_combined": [best_estimator_combined],

    "MLPC"       : [MLPClassifier(random_state=42)],  ##default
    "MLP_morpho"  : [best_estimator_morpho_mlpc ],
    "MLP_struct"  : [best_estimator_struct_mlpc],
    "MLP_combined": [best_estimator_combined_mlpc]
}

def evaluate_model(y_test,y_pred):
    
    '''
    Compute and asses a model with acc balanced acc and f1 metrics 
        Parameters : 
            y_test: test to predict 
            y_pred: pred made by the model
        Returns : 
            classification report 
    '''    
    
    ba = balanced_accuracy_score(y_test,y_pred)
    mcc = matthews_corrcoef(y_pred=y_pred, y_true=y_test)

    print(f'Evaluation of predicition made on Test set : ')
    print(f'Metrics to evaluate the model:\n Balanced accuracy : {ba*100:.2f} % ,\n MCC : {mcc:.3f}.')
    print(f'Summary metrics in a cross table:\n')
    print(classification_report(y_test, y_pred, zero_division=0))

def make_predictions(models_dict ,selected_model,X_train,X_test,y_test,y_train) :

    '''
    Function to call to evaluate the model and do a cross val 
        Parameters : 
            models_dict (dictionary): dict of defined architecture and its parameters
            selected_model (str): name of the model to choose
            y_test : vector to predict either morphological or structural fingerprint or both 
    '''      

            
    if selected_model in models_dict :
        model = models_dict[selected_model][0]

    #learn by fitting the model on the trainset 
    model.fit(X_train,y_train) 
    
    #compute prediction on test set 
    predictions = model.predict(X_test)
    evaluate_model(y_test,predictions) 
    return model



## Make prediction on test set 

1. Morphological fingerprint RF

In [ ]:
default_morpho_rf = make_predictions(models,'RF',X_train,X_test,y_test_activities,y_train_activities)


In [ ]:
morpho_model_rf = make_predictions(models,'RF_morpho',X_train,X_test,y_test_activities,y_train_activities)

2. Morphological fingerprint MLPC

In [ ]:
default_morpho_MLPC = make_predictions(models,'MLPC',X_train,X_test,y_test_activities,y_train_activities)

In [ ]:
morpho_MLPC = make_predictions(models,'MLP_morpho',X_train,X_test,y_test_activities,y_train_activities)

1.B Structural fingerprint

In [ ]:
default_struct_rf = make_predictions(models,'RF',X_train_struct_fps,X_test_struct_fps,y_test_activities,y_train_activities)

In [ ]:
struct_model_rf = make_predictions(models,'RF_struct',X_train_struct_fps,X_test_struct_fps,y_test_activities,y_train_activities)

2.B Structural fingerprint

In [ ]:
default_morpho_MLPC = make_predictions(models,'MLPC',X_train_struct_fps,X_test_struct_fps,y_test_activities,y_train_activities)

In [ ]:
morpho_MLPC = make_predictions(models,'MLP_struct',X_train_struct_fps,X_test_struct_fps,y_test_activities,y_train_activities)

3. Combined 

First, before running model on combined fingerprint we will append the structural and morphological fineprrint using 

In [ ]:
combined_model_rf = make_predictions(models,'RF_combined',combined_train,combined_test,y_test_activities,y_train_activities)

In [ ]:
combined_model_rf_default= make_predictions(models,'RF',combined_train,combined_test,y_test_activities,y_train_activities)

In [ ]:
combined_model_MLP = make_predictions(models,'MLP_combined',combined_train,combined_test,y_test_activities,y_train_activities)

In [ ]:
combined_model_MLP_default = make_predictions(models,'MLPC',combined_train,combined_test,y_test_activities,y_train_activities)